# AI-Powered Academic Research Workshop
## Using OpenAI API for Literature Reviews and Paper Analysis

This notebook provides practical examples of using AI tools for academic research workflows.

### Learning goals
- Understand what an API is and how to call it from Python
- Learn to design and customize prompt templates for repeatable tasks
- Use simple `for` loops to automate common research workflows
- Structure outputs (JSON) so they are easy to store, filter, and analyze

### What you’ll build
- A single-paper analyzer that extracts key fields into JSON
- A batch processor that loops over many papers with a progress bar
- A relevance rater that scores papers against your research question
- A short literature synthesis generated from multiple items

### How to use this notebook
- Run cells top-to-bottom the first time
- Edit prompts in the `Prompt templates` section; re-run only the cells that depend on them
- Use the `Loop patterns` section to demonstrate how iteration works step by step

## New to APIs? Quick primer

- **What is an API?** A way for your code to "ask" another service (like an AI model) to do something and return a result — like ordering from a menu and the kitchen delivers the dish.
- **Why use it here?** We send a text prompt to an AI model and get a text response back.
- **What do you need?**
  - An account/key to prove who you are 
  - A small bit of setup to let Python talk to the service
- **Costs/tokens:** Models bill per token (roughly word pieces). We'll show how to estimate and keep requests efficient.

### How a request works (mental model)
1) You prepare a prompt (your instructions)
2) Your code sends it to the API endpoint
3) The service runs the model and returns a response
4) You parse the response and use it (print, save, analyze)

```
You  →  API endpoint  →  Model runs  →  Response back to you
```

### What “messages” mean here
- We send both a short "system" message (role/context) and a "user" message (your prompt)
- The model uses both to generate the reply


## First-run checklist (you can copy-paste this into your notes)

1) Create an account and an API key (your "library card").
2) Store the key as an environment variable named `OPENAI_API_KEY`.
   - macOS (temporary for current Terminal session):
     - `export OPENAI_API_KEY="sk-..."`
   - macOS (persistent in `~/.zshrc`):
     - Add this line: `export OPENAI_API_KEY="sk-..."` and then run `source ~/.zshrc`
3) Run the connection test cell to verify access.
4) Run the examples; adjust `DEFAULT_MODEL` if desired.
5) If batching many items, estimate tokens first to stay within budget.


## Part 1: Setting Up - Loading API Keys Securely

First, we'll load the necessary libraries and set up secure API key management.

In [ ]:
# Install required packages (run once)
# !pip install openai pandas requests PyPDF2 tqdm python-dotenv

# If you encounter issues, try:
# !pip install openai==0.28.1  # For older stable version
# or
# !pip install --upgrade openai  # For latest version

In [1]:
import os
import getpass
import pandas as pd
import json

# For older OpenAI library (< 1.0)
import openai

# Get API key securely
if os.getenv('OPENAI_API_KEY'):
    openai.api_key = os.getenv('OPENAI_API_KEY')
    print("✅ Using API key from environment")
else:
    api_key = getpass.getpass(prompt="Enter your OpenAI API key: ")
    openai.api_key = api_key
    print("✅ API key set")

print("Ready to make API calls!")

✅ API key set
Ready to make API calls!


## Part 2: Basic API Query - Testing the Connection

We’ll send a simple question and print the model’s reply.

What to notice:
- We pass a short `system` role for context (tone/role of the assistant)
- The `temperature` controls randomness (0 = consistent; 1 = creative)
- You can change `model` to a different one if available

In [2]:
def simple_query(prompt, model="gpt-3.5-turbo", temperature=0.7):
    """
    Send a simple query to the OpenAI API.
    
    Args:
        prompt: The question or prompt to send
        model: The model to use (gpt-3.5-turbo is cost-effective)
        temperature: Controls randomness (0=deterministic, 1=creative)
    
    Returns:
        The model's response as a string
    """
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful academic research assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=temperature
    )
    
    return response['choices'][0]['message']['content']

# Test the connection
test_prompt = "What are the key components of a systematic literature review?"
response = simple_query(test_prompt)
print("Response from AI:")
print(response)

Response from AI:
A systematic literature review typically involves the following key components:

1. Research question or objective: Clearly defining the research question or objective that the review aims to address is essential. This helps guide the search strategy and selection criteria for including studies.

2. Search strategy: Developing a comprehensive search strategy to identify relevant studies is crucial. This involves identifying relevant databases, keywords, and search terms to ensure a thorough and systematic search.

3. Study selection criteria: Establishing specific criteria for selecting studies to be included in the review helps ensure that the review is systematic and focused. These criteria may include factors such as publication date, study design, population characteristics, and outcome measures.

4. Data extraction: Systematically extracting relevant data from the selected studies is a key component of a systematic literature review. This typically involves creat

## Part 3: Single Paper Review - Extracting Key Information

We’ll extract a consistent JSON structure from a single paper.

Why JSON?
- Easy to parse in Python, save to CSV, or load into a database
- Consistent fields make downstream analysis simpler (filter, aggregate, visualize)

Prompt tips:
- Say "Return ONLY valid JSON" to avoid extra text
- Enumerate keys you want back (e.g., `research_question`, `methodology`, `key_findings`)

In [3]:
def analyze_single_paper(paper_title, paper_abstract):
    """
    Analyze a single academic paper and extract key information.
    
    Args:
        paper_title: The title of the paper
        paper_abstract: The abstract of the paper
    
    Returns:
        A dictionary with extracted information
    """
    
    prompt = f"""
    Please analyze this academic paper:
    
    Title: {paper_title}
    
    Abstract: {paper_abstract}
    
    Extract the following in JSON format:
    - research_question: Main research question
    - methodology: Brief description of methods
    - key_findings: List of 3-5 main findings
    - implications: Practical implications
    - limitations: Any limitations mentioned
    
    Return ONLY the JSON object.
    """
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an academic paper analyst. Return only JSON."},
            {"role": "user", "content": prompt}
        ],
        temperature=0  # Use 0 for consistent extraction
    )
    
    result_text = response['choices'][0]['message']['content'].strip()
    
    # Clean up JSON if needed
    if result_text.startswith("```json"):
        result_text = result_text[7:-3]
    
    try:
        return json.loads(result_text)
    except:
        return {"error": "Failed to parse response", "raw": result_text}

# Example usage
sample_title = "The Impact of AI Tools on Academic Research Productivity"
sample_abstract = """
This study examines the impact of AI tools on academic research productivity. 
Through surveys (n=500) with researchers, we find that AI tools reduce time 
spent on literature reviews by 40% and data analysis by 35%. However, concerns 
about accuracy and over-reliance on AI emerge as key challenges.
"""

result = analyze_single_paper(sample_title, sample_abstract)
print("Analysis Result:")
print(json.dumps(result, indent=2))

Analysis Result:
{
  "research_question": "To what extent do AI tools impact academic research productivity?",
  "methodology": "Surveys were conducted with 500 researchers to gather data on the use of AI tools in academic research.",
  "key_findings": [
    "AI tools reduce time spent on literature reviews by 40%.",
    "AI tools reduce time spent on data analysis by 35%.",
    "Concerns about accuracy and over-reliance on AI are key challenges."
  ],
  "implications": "The findings suggest that AI tools can significantly improve research productivity but also raise concerns that need to be addressed.",
  "limitations": "The study relies on self-reported data from researchers and may be subject to response bias."
}


## Part 4: Batch Processing - Analyzing Multiple Papers

We’ll loop over many items and collect results.

What to notice:
- A `for` loop iterates paper-by-paper
- `tqdm` adds a progress bar so students see progress and timing
- `try/except` ensures one failure doesn’t stop the whole batch

Teaching tip:
- Start with 2–3 items to keep it fast, then scale up

In [4]:
from tqdm import tqdm

def process_paper_batch(papers_list):
    """
    Process multiple papers in a loop.
    
    Args:
        papers_list: List of dictionaries with 'title' and 'abstract'
    
    Returns:
        List of analysis results
    """
    
    results = []
    
    # Process each paper with a progress bar
    for paper in tqdm(papers_list, desc="Processing papers"):
        try:
            result = analyze_single_paper(
                paper_title=paper['title'],
                paper_abstract=paper['abstract']
            )
            results.append(result)
        except Exception as e:
            results.append({"error": str(e)})
    
    return results

# Sample papers for demonstration
sample_papers = [
    {
        'title': 'Machine Learning in Healthcare',
        'abstract': 'This review examines ML applications in healthcare diagnosis and treatment.'
    },
    {
        'title': 'Climate Change and Economic Growth',
        'abstract': 'Using panel data from 150 countries, we study climate impacts on GDP.'
    },
    {
        'title': 'Digital Divide in Education',
        'abstract': 'COVID-19 exposed disparities in digital learning access across demographics.'
    }
]

# Process the batch
print("Processing batch of papers...")
results = process_paper_batch(sample_papers)

# Display results
for i, result in enumerate(results):
    print(f"\nPaper {i+1}:")
    print(json.dumps(result, indent=2))

Processing batch of papers...


Processing papers: 100%|██████████| 3/3 [00:05<00:00,  1.78s/it]


Paper 1:
{
  "research_question": "To examine machine learning applications in healthcare diagnosis and treatment",
  "methodology": "Review of existing literature and case studies on machine learning in healthcare",
  "key_findings": [
    "Machine learning can improve diagnostic accuracy in healthcare",
    "Machine learning can assist in personalized treatment plans for patients",
    "Machine learning can help in predicting patient outcomes"
  ],
  "implications": "The findings suggest that integrating machine learning in healthcare can lead to more accurate diagnoses and personalized treatment plans, ultimately improving patient outcomes.",
  "limitations": "One limitation mentioned is the need for large and diverse datasets for training machine learning models in healthcare."
}

Paper 2:
{
  "research_question": "To study climate impacts on GDP",
  "methodology": "Panel data analysis from 150 countries",
  "key_findings": [
    "Climate change has a negative impact on economic g

## Part 5: Real-World Example - Meta-Analysis of NBER Papers on AI

Let's analyze actual NBER working papers about AI and generative models using our Python modules.

In [ ]:
print("\nStep 3: Saving results to CSV file...")

# Save the complete meta-analysis to CSV
output_filename = 'nber_ai_papers_meta_analysis.csv'
meta_analysis_df.to_csv(output_filename, index=False)

print(f"✅ Results saved to: {output_filename}")
print(f"   Total papers analyzed: {len(meta_analysis_df)}")
print(f"   Columns in dataset: {len(meta_analysis_df.columns)}")

# Display summary statistics
print("\n" + "="*60)
print("SUMMARY STATISTICS:")
print("="*60)

# Show key columns available in the dataset
key_columns = ['Paper_Title', 'Author(s)', 'Year', 'Research_Question', 
               'Methodology', 'Main_Results', 'Relevance_Likert_Scale']

print("\nKey columns in the dataset:")
for col in key_columns:
    if col in meta_analysis_df.columns:
        print(f"  ✓ {col}")

# Calculate average relevance score if available
if 'Relevance_Likert_Scale' in meta_analysis_df.columns:
    avg_relevance = meta_analysis_df['Relevance_Likert_Scale'].mean()
    print(f"\nAverage Relevance Score: {avg_relevance:.2f}/5")

print("\n💡 You can now open the CSV file in Excel or pandas for further analysis!")
print("   Try: df = pd.read_csv('nber_ai_papers_meta_analysis.csv')")

## Part 7: Saving Results to CSV

In [ ]:
# Import our custom modules for paper processing
import sys
sys.path.append('/Users/patrickhealy/Documents/phd_ai_workshop')

from paper_processing import process_papers
from meta_analysis_processing import process_papers_in_parallel

# List of 10 recent NBER papers on AI and research (2024)
nber_papers_urls = [
    "https://www.nber.org/system/files/working_papers/w33198/w33198.pdf",  # Generative AI for Economic Research
    "https://www.nber.org/system/files/working_papers/w32966/w32966.pdf",  # Rapid Adoption of Generative AI
    "https://www.nber.org/system/files/working_papers/w32980/w32980.pdf",  # Economic Policy Challenges for AI
    "https://www.nber.org/system/files/working_papers/w33139/w33139.pdf",  # Concentrating Intelligence: AI Market Structure
    "https://www.nber.org/system/files/working_papers/w32487/w32487.pdf",  # Simple Macroeconomics of AI
    "https://www.nber.org/system/files/working_papers/w32474/w32474.pdf",  # Old Moats for New Models
    "https://www.nber.org/system/files/working_papers/w31161/w31161.pdf",  # Generative AI at Work
    "https://www.nber.org/system/files/working_papers/w31222/w31222.pdf",  # Generative AI and Firm Values
    "https://www.nber.org/system/files/working_papers/w32047/w32047.pdf",  # Generative AI's Impact on Skilled Work
    "https://www.nber.org/system/files/working_papers/w32884/w32884.pdf",  # AI and Future of Work
]

print("Step 1: Downloading and extracting text from NBER papers...")
print(f"Processing {len(nber_papers_urls)} papers on AI and generative models\n")

# Download and process the papers
papers_df = process_papers(nber_papers_urls)

print(f"\n✅ Successfully extracted text from {len(papers_df)} papers")
print("\nSample from first paper:")
print(f"Title/Abstract preview: {papers_df.iloc[0]['Title_Abstract'][:300]}...")

## Part 6: Performing Meta-Analysis with GPT-4o-mini

In [15]:
print("\nStep 2: Performing meta-analysis using GPT-4o-mini...")
print("Extracting structured information from each paper:\n")

# Create an OpenAI client adapter that matches what meta_analysis_processing.py expects
class OpenAIClient:
    def __init__(self):
        self.api_key = openai.api_key
        self.chat = self  # The module expects client.chat
        self.completions = self  # The module expects client.chat.completions
    
    def create(self, **kwargs):
        # Convert the call to the old OpenAI API format
        return openai.ChatCompletion.create(**kwargs)

# Initialize client
client = OpenAIClient()

# Perform meta-analysis using GPT-4o-mini (cost-effective model)
meta_analysis_df = process_papers_in_parallel(
    papers_df, 
    client=client,
    chunk_size=1,       # Process one paper at a time for clarity
    n_workers=5,        # Use 5 parallel workers for speed
    temperature=0.3     # Low temperature for consistent extraction
)

print(f"\n✅ Meta-analysis complete for {len(meta_analysis_df)} papers")

# Display first paper's analysis as example
if len(meta_analysis_df) > 0:
    first = meta_analysis_df.iloc[0]
    print("\n" + "="*60)
    print("EXAMPLE OUTPUT - First Paper Analysis:")
    print("="*60)
    print(f"Title: {first.get('Paper_Title', 'N/A')}")
    print(f"Authors: {first.get('Author(s)', 'N/A')}")
    print(f"Year: {first.get('Year', 'N/A')}")
    print(f"\nResearch Question: {first.get('Research_Question', 'N/A')[:200]}...")
    print(f"\nMethodology: {first.get('Methodology', 'N/A')[:200]}...")
    print(f"\nKey Findings: {first.get('Main_Results', 'N/A')[:200]}...")


Step 2: Performing meta-analysis using GPT-4o-mini...
Extracting structured information from each paper:



Processing chunks in parallel: 100%|██████████| 10/10 [00:16<00:00,  1.62s/it]


✅ Meta-analysis complete for 10 papers

EXAMPLE OUTPUT - First Paper Analysis:
Title: Generative AI for Economic Research: LLMs Learn to Collaborate and Reason
Authors: Anton Korinek
Year: 2024

Research Question: How can economists utilize advancements in large language models (LLMs) to enhance their research productivity?...

Methodology: The paper provides a practical guide and overview of advancements in LLMs, focusing on their reasoning capabilities, collaborative workspaces, and improvements in internet search....

Key Findings: LLMs have significantly improved in processing speed, cost efficiency, and reasoning capabilities, leading to enhanced productivity for economists....


In [16]:
meta_analysis_df

,Paper URL,Title_Abstract,Conclusion,Paper_ID,Author(s),Year,Paper_Title,Journal_Specific_ID,Research_Question,Methodology,...,Sample_Population,Main_Results,Effect_Size,Key_Statistical_Measures,Data_Source,Limitations,Implications_for_Policy,Open_Questions,Relevance_Likert_Scale,Relevance_Reasons
0,https://www.nber.org/system/files/working_pape...,NBER WORKING PAPER SERIES\nGENERATIVE AI FOR E...,Figure 1: Decline in op erating costs and qual...,P001,Anton Korinek,2024,Generative AI for Economic Research: LLMs Lear...,NBER Working Paper No. 33198,How can economists utilize advancements in lar...,The paper provides a practical guide and overv...,...,Not specified; discusses advancements in AI ap...,LLMs have significantly improved in processing...,Not quantified; qualitative improvements in pr...,Not applicable; the paper is more of a concept...,Literature review and synthesis of advancement...,The paper does not provide empirical data or q...,Incorporating LLMs into economic research can ...,Future research could explore the specific met...,5,The paper directly addresses the use of AI too...
1,https://www.nber.org/system/files/working_pape...,NBER WORKING PAPER SERIES\nTHE RAPID ADOPTION ...,We can then write the percent increase in aggr...,P001,"Alexander Bick, Adam Blandin, David J. Deming",2024,The Rapid Adoption of Generative AI,NBER Working Paper No. 32966,How does the speed and intensity of generative...,The paper utilizes nationally representative s...,...,Nationally representative U.S. surveys with a ...,Approximately 40% of the U.S. population uses ...,Each hour spent using generative AI increases ...,Estimates of productivity gains with a potenti...,"Real-Time Population Survey (RPS), a nationall...","The study relies on self-reported data, which ...",The rapid adoption of generative AI suggests t...,Future research should track the scale and int...,3,The paper discusses generative AI's adoption a...
2,https://www.nber.org/system/files/working_pape...,NBER WORKING PAPER SERIES\nECONOMIC POLICY CHA...,15 \n share their emotional pains. They may pr...,P001,Anton Korinek,2024,Economic Policy Challenges for the Age of AI,http://www.nber.org/papers/w32980,What challenges will transformative advances i...,The paper is a conceptual analysis that synthe...,...,Theoretical framework based on existing litera...,Identifies eight key challenges for economic p...,Not applicable as the paper does not provide q...,Not applicable.,Literature review and theoretical insights.,The paper is conceptual and does not provide e...,Policies must adapt to ensure equitable distri...,How can policymakers effectively implement new...,3,The paper discusses the implications of AI on ...
3,https://www.nber.org/system/files/working_pape...,NBER WORKING PAPER SERIES\nCONCENTRATING INTEL...,be right. This may lead nurses to err ...,P001,"Anton Korinek, Jai Vipra",2024,Concentrating Intelligence: Scaling and Market...,NBER Working Paper No. 33139,How does the evolving structure and competitio...,The paper employs a theoretical analysis of ma...,...,Theoretical framework based on existing litera...,The analysis suggests that significant economi...,Not applicable as the study is theoretical.,Not applicable as the study does not present e...,Literature review and theoretical constructs.,Theoretical nature limits empirical validation...,Policy remedies are necessary to maintain a co...,What specific regulatory frameworks can effect...,3,"The paper discusses market dynamics in AI, whi..."
4,https://www.nber.org/system/files/working_pape...,NBER WORKING PAPER SERIES\nTHE SIMPLE MACROECO...,"affected groups, because AI-exposed tasks are ...",P001,Daron Acemoglu,2024,The Simple Macroeconomics of AI,NBER Working Paper No. 32487,What are the macroeconomic implications of new...,The paper employs a task-based model to evalua...,...,Estimates based on existing data on AI exposur...,AI's macroeconomic effects are estimated to be...,Predicted TFP gains are less tha

## Loop patterns: minimal, step-by-step, and vectorized

These examples separate the loop logic from prompts and API calls. They print intuitive, teaching-friendly outputs:

- What is being iterated
- Which prompt is being used
- What the model returned (pretty-printed)
- How to switch models

What each function demonstrates:
- `loop_minimal`: the smallest working `for` loop + prompt
- `loop_step_by_step`: prints each step, the prompt preview, and the result
- `loop_vectorized_single_prompt`: combines multiple items into a single request for efficiency


## Glossary (1-minute read)

- **API**: A way for programs to talk to services
- **API key**: Your ID card for the service
- **Prompt**: The instruction or question you send to the model
- **Tokens**: Chunks of text used for billing/limits
- **Temperature**: Controls randomness (lower = more consistent)
- **Rate limit**: The maximum requests allowed in a time window (handle 429 errors with short delays)


## Troubleshooting and privacy notes

- 401 Unauthorized: check `OPENAI_API_KEY` is set correctly
- 429 Rate limit: wait briefly and retry; reduce request rate or batch items
- 500 Server error: retry after a short delay; if persistent, try later
- Validate outputs: treat AI responses as drafts to review, not truth
- Avoid sending sensitive data; anonymize where possible
- For JSON outputs, keep prompts strict and add fallback parsing if needed
